In [18]:
import os
os.chdir("../")
os.getcwd()

'c:\\Users\\hp\\Documents\\iNeuron_intern\\covid_mask'

In [2]:
from covid_md.utils import read_yaml, create_directories
from covid_md.constant import *
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

In [27]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataTrainingConfig:
    root_dir: Path
    data_path: Path
    test_size: float
    monitor: str
    verbose: int
    save_best_only: bool
    mode: str
    epochs: int
    validation_split: float
    model: str

In [28]:
class ConfigurationManager:
    def __init__(self, config_filepath= CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def get_data_training_config(self)-> DataTrainingConfig:
        config= self.config.data_training
        params= self.params.training
        create_directories([config.root_dir])
        data_training_config= DataTrainingConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            test_size= params.test_size,
            monitor= params.monitor,
            verbose= params.verbose,
            save_best_only= params.save_best_only,
            mode= params.mode,
            epochs= params.epochs,
            validation_split= params.validation_split,
            model= params.model
        )
        return data_training_config

In [41]:
class DataTraining:
    def __init__(self,config: DataTrainingConfig):
        self.config= config
    def load_data(self):
        os.chdir(self.config.data_path)
        data= np.load('data.npy')
        target= np.load('target.npy')
        os.chdir("../../")
        return data, target
    def training_skeleton(self,data):
        model= Sequential()

        model.add(Conv2D(200,(3,3),input_shape= data.shape[1:]))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))

        model.add(Conv2D(100,(3,3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))

        model.add(Flatten())
        model.add(Dropout(0.5))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(2, activation= 'softmax'))

        model.compile(loss= 'categorical_crossentropy', optimizer= 'adam', metrics= ['accuracy'])
        return model
    def split_data(self, data, target):
        test_size= self.config.test_size
        train_data,test_data,train_target,test_target= train_test_split(data,target, test_size= test_size)
        return train_data,test_data,train_target,test_target
    def checkpoint(self):
        model= self.config.model
        monitor= self.config.monitor
        verbose= self.config.verbose
        best_model= self.config.save_best_only
        mode= self.config.mode
        cp= ModelCheckpoint(model, monitor= monitor, verbose= verbose, save_best_only= best_model, mode= mode)
        return cp
    def final_training(self,train_data, train_target,checkpoint,model):
        epochs= self.config.epochs
        validation_split= self.config.validation_split
        os.chdir(self.config.root_dir)
        history= model.fit(train_data, train_target, epochs= epochs, callbacks= [checkpoint], validation_split= validation_split)
        os.chdir("../../")
        return history

In [43]:
try:
    config= ConfigurationManager()
    data_training_config= config.get_data_training_config()
    data_training= DataTraining(config= data_training_config)
    data,target= data_training.load_data()
    model= data_training.training_skeleton(data)
    train_data,test_data,train_target,test_target= data_training.split_data(data, target)
    checkpoint= data_training.checkpoint()
    history= data_training.final_training(train_data, train_target, checkpoint, model)
except Exception as e:
    raise e

Epoch 1/20
2/2 [==============================] - 81s 40s/step - loss: 0.8107 - accuracy: 0.5200 - val_loss: 0.7033 - val_accuracy: 0.6154
Epoch 2/20
2/2 [==============================] - 12s 10s/step - loss: 0.7978 - accuracy: 0.5000 - val_loss: 0.6972 - val_accuracy: 0.3846
Epoch 3/20
2/2 [==============================] - 4s 2s/step - loss: 0.6982 - accuracy: 0.5000 - val_loss: 0.7032 - val_accuracy: 0.3846
Epoch 4/20
2/2 [==============================] - 8s 5s/step - loss: 0.6877 - accuracy: 0.5000 - val_loss: 0.6701 - val_accuracy: 0.6154
Epoch 5/20
2/2 [==============================] - 12s 10s/step - loss: 0.6889 - accuracy: 0.5000 - val_loss: 0.6607 - val_accuracy: 0.6154
Epoch 6/20
2/2 [==============================] - 5s 2s/step - loss: 0.6645 - accuracy: 0.6800 - val_loss: 0.6953 - val_accuracy: 0.3846
Epoch 7/20
2/2 [==============================] - 8s 5s/step - loss: 0.6775 - accuracy: 0.5000 - val_loss: 0.6269 - val_accuracy: 1.0000
Epoch 8/20
2/2 [===================

In [49]:
from sklearn.metrics import mean_squared_error
predict= model.predict(test_data)
mean_squared_error(predict, test_target)

1/1 [==============================] - 0s 222ms/step


0.020628016

In [50]:
model.evaluate(test_data,test_target)

1/1 [==============================] - 2s 2s/step - loss: 0.1344 - accuracy: 1.0000


[0.13444918394088745, 1.0]